In [ ]:
%%capture
!git clone https://github.com/smfm-project/biota.git
!pip install scikit-image
!pip uninstall biota -y
!pip install biota/

In [1]:
import os
import getpass
from pathlib import Path
from functools import partial

import biota
from biota import download as dw

from ipyleaflet import Marker
from ipywidgets import Output, Layout, HTML
from traitlets import Int, Bool, Float, link, Unicode
import ipyvuetify as v
from sepal_ui import mapping as m
from sepal_ui import sepalwidgets as sw;

HTML(value='\n<style>\n.leaflet-pane {\n    z-index : 2 !important;\n}\n.leaflet-top, .leaflet-bottom {\n    z…

ResizeTrigger()

In [ ]:
from ipywidgets import HTML

In [ ]:
STYLES = """
<style>
.output_text pre, .widget-html{
    color : #fff !important;
}

</style>
"""

_ = display(HTML(STYLES));

In [82]:
HEADER_TITLE = "SEPAL BIOTA"
HEADER_TEXT = """
The BIOmass Tool for Alos (BIOTA) was developed by LTS International and the University of Edinburgh to calculate avove-ground biomass from L-band satallite data in dry forests and savanhas, as well as biomass change and forest degradation.
"""

In [95]:
class UI(v.Layout):
    
    USER = getpass.getuser()
    PARAMETER_FILE = os.path.join('/home', USER, 'SMFM_biota/biota/cfg/McNicol2018.csv')

    # Initial widgets
    lat = Float(0).tag(sync=True)
    lon = Float(75).tag(sync=True)
    year_1 = Unicode().tag(sync=True)
    year_2 = Unicode().tag(sync=True)
    large_tile = Bool(True).tag(sync=True)
    grid = Int(5).tag(sync=True)
    
    # Process widgets
    forest_p = Bool(True).tag(sync=True)
    forest_ch = Bool(True).tag(sync=True)
    gamma0 = Bool(True).tag(sync=True)
    biomass = Bool(True).tag(sync=True)
    forest_cv = Bool(True).tag(sync=True)
    

    
    def __init__(self, map_=None, **kwargs):
        
        # Widget classes
        self.class_ = "flex-column pa-2"
        self.row = True
        self.xs12 = True
        
        super().__init__(**kwargs)
        
        # Set up variables
        
        self.map_ = map_
        
        # Set-up workspace
        self._workspace()

        # Widgets
        self.progress = Output()
#         layout=Layout(height='80px')
        
        self.w_lat = v.TextField(disabled=True, label='Latitude',v_model=self.lat,)
        self.w_lon = v.TextField(disabled=True, label='Longitude', v_model=self.lon,)
        self.w_year_1 = v.TextField(label='Year 1', v_model=self.year_1,)
        self.w_year_2 = v.TextField(label='Year 2', v_model=self.year_2,)
        self.w_lg_tile = v.Checkbox(label='Large Tile', v_model=self.large_tile)
        self.w_grid = v.RadioGroup(v_model=self.grid,children=[
            v.Radio(label="1x1 grid", value=1),v.Radio(label="5x5 grid", value=5)
        ])
        
        # Process widgets
        self.w_forest_p = v.Checkbox(label='Forest property', class_='pl-5', v_model=self.forest_p)
        self.w_forest_ch = v.Checkbox(label='Forest change', class_='pl-5', v_model=self.forest_ch)
        self.w_gamma0 = v.Checkbox(label='Gamma0', class_='pl-5', v_model=self.gamma0)
        self.w_biomass = v.Checkbox(label='Biomass', class_='pl-5', v_model=self.biomass)
        self.w_forest_cv = v.Checkbox(label='Forest cover', class_='pl-5', v_model=self.forest_cv)
        self.w_download = sw.Btn('Download images', class_='pl-5')
            
        self.w_alert = sw.Alert()
        self.w_download = sw.Btn('Download images', class_='pl-5')
        
        

        link((self.w_lon, 'v_model'), (self, 'lon'))
        link((self.w_lat, 'v_model'), (self, 'lat'))
        link((self.w_year_1, 'v_model'), (self, 'year_1'))
        link((self.w_year_2, 'v_model'), (self, 'year_2'))
        link((self.w_lg_tile, 'v_model'), (self, 'large_tile'))
        link((self.w_grid, 'v_model'), (self, 'grid'))

        link((self.w_forest_p, 'v_model'), (self, 'forest_p'))
        link((self.w_forest_ch, 'v_model'), (self, 'forest_ch'))
        link((self.w_gamma0, 'v_model'), (self, 'gamma0'))
        link((self.w_biomass, 'v_model'), (self, 'biomass'))
        link((self.w_forest_cv, 'v_model'), (self, 'forest_cv'))

        # Events
        self.w_download.on_event('click', self._download)
#         class_="d-flex flex-row ", xs12=True, md6=True,
        self.children = [
            
            v.Card(children=[
                v.CardTitle(children=[HEADER_TITLE]), 
                v.CardText(children=[HEADER_TEXT])
            ]),
            
            v.Row(class_="d-flex flex-row ", xs12=True, md6=True,
               children=[
                    v.Col(
                        children=[
                            v.Card(class_='pa-4',children=[
                                self.w_lon, 
                                self.w_lat,
                                self.w_year_1,
                                self.w_year_2,
                                self.w_lg_tile,
                                self.w_grid,
                                self.w_download
                            ])

                        ]
                    ),
                    v.Col(
                        children=[
                            v.Card(class_='pa-2 justify-center', children=[
                                map_
                            ])
                        ]
                    ),
            ]),

            # Second row
            # Progress bar
            v.Card(class_="flex-row pa-2 mb-3", children=[
                self.w_alert,
                self.progress,

            ]),
            
            # Third row
            # Progress bar
            v.Card(class_="d-flex flex-row", children=[
                v.Col(
                    children=[
                            self.w_forest_p,
                            self.w_forest_ch,
                    ]
                ),
                v.Col(
                    children=[
                            self.w_gamma0,
                            self.w_biomass,
                    ]
                ),
                v.Col(
                    children=[
                            self.w_forest_cv,
                    ]
                ),
            ]
            )
        ]
    
    def _round(self, x, base=5):
        return base * round(x/base)
    
    def _workspace(self):
        """ Creates the workspace necessary to store the data

        return:
            Returns environment Paths

        """

        base_dir = Path(os.path.join('/home', self.USER))

        root_dir = base_dir/'smfm'
        data_dir = root_dir/'data'
        output_dir = root_dir/'outputs'

        root_dir.mkdir(parents=True, exist_ok=True)
        data_dir.mkdir(parents=True, exist_ok=True)
        output_dir.mkdir(parents=True, exist_ok=True)

        self.root_dir = root_dir
        self.data_dir  = data_dir
        self.output_dir = output_dir
    
    
    def _download(self, widget, event, data):
        
        self.w_download.toggle_loading()
        with self.progress:
            
            try:
                self.progress.clear_output()
                self.w_alert.reset()

                years = [int(year) for year in [self.year_1, self.year_2] if year]
                lat = self._round(self.lat, base=self.grid)
                lon = self._round(self.lon, base=self.grid)

                if not years:
                    self.w_alert.add_msg('Please select at least one year.', type_='error')

                for y in years:
                    dw.download(lat, 
                                lon, 
                                y, 
                                large_tile=self.large_tile, 
                                output_dir=self.data_dir, 
                                verbose=True)
            except Exception as e:
                self.w_alert.add_msg(f'{e}', type_='error')
                
        self.w_download.toggle_loading()

    def _decompress(self):

        tar_files = list(self.data_dir.glob('*.tar.gz'))

        for tar in tar_files:
            if not os.path.exists(os.path.join(self.data_dir, tar.name[:-7])):
                download.decompress(tar)

In [96]:
def _return_coordinates(self, **kwargs):
    if kwargs.get('type') == 'click':

        # Remove markdown if there is one
        map_.remove_last_layer()

        lat, lon = kwargs.get('coordinates')

        map_.add_layer(Marker(location=kwargs.get('coordinates')))
        ui.lat = round(lat,2)
        ui.lon = round(lon,2)
# Map
map_ = m.SepalMap()
# map_.layout.height='400px'
# map_.layout.width='400px'
ui = UI(map_=map_)
map_.on_interaction(partial(_return_coordinates, ui))

In [97]:
ui

UI(children=[Card(children=[CardTitle(children=['SEPAL BIOTA']), CardText(children=['\nThe BIOmass Tool for Al…

In [81]:
v.Card(children=[
    v.CardTitle(children=['Hola']),
    v.CardSubtitle(children=['Hola']),
    v.CardText(children=['This is the content']),
])

Card(children=[CardTitle(children=['Hola']), CardSubtitle(children=['Hola']), CardText(children=['This is the …

In [ ]:
m_ = m.SepalMap()

In [ ]:
v.Layout(class_="flex-column pa-2", children=[
    
    # First row
    # Download Inputs
    v.Row(class_="d-flex flex-row ", xs12=True, md6=True,
           children=[
                v.Col(
                    children=[
                        v.Card(class_='pa-2',children=[
                            v.TextField(disabled=True, class_='d-flex', label='Latitude'),
                            v.TextField(disabled=True, class_='d-flex', label='Latitude'),
                        ])

                    ]
                ),
                v.Col(
                    children=[
                        v.Card(class_='pa-2 justify-center', children=[
                            map_
                        ])

                    ]
                ),
        ]),
    
    # Second row
    # Process selector
    v.Card(class_="flex-row", children=[
        v.Input(children=['items3'])
    ]),

])

In [ ]:
ou = Output(class_="v-label theme--dark")

In [ ]:
with ou:
    ou.clear_output()
#     x = sw.Alert().add_msg('Holi')
#     display(x)
    pbar = tqdm(total=100)
    for i in range(10):
        sleep(0.1)
        pbar.update(10)
    pbar.close()

In [ ]:
v.Layout(children=[ou, sw.Markdown(mkd_str="<p>hola</p>")])

In [ ]:
Output?

In [57]:
tile_2018 = biota.LoadTile(str(ui.data_dir), 
                           0, 
                           -75, 
                           2018,
                           parameter_file = ui.PARAMETER_FILE,
                           lee_filter = True, 
                           forest_threshold = 15., 
                           area_threshold = 1, 
                           output_dir = str(ui.output_dir))

In [ ]:
# Define latitude and longitude
latitude = 0
longitude = -75

# Load the ALOS tile for 2007...
tile_2018 = biota.LoadTile(data_dir, latitude, longitude, 2018, lee_filter = True, forest_threshold = 15., area_threshold = 1, output_dir = output_dir)

# ...and for 2017
#tile_2017 = biota.LoadTile(data_dir, latitude, longitude, 2017, lee_filter = True, forest_threshold = 15., area_threshold = 1, output_dir = output_dir)

# ...and for 2015
tile_2015 = biota.LoadTile(data_dir, latitude, longitude, 2015, lee_filter = True, forest_threshold = 15., area_threshold = 1, output_dir = output_dir)

# ...and for 2010
#tile_2010 = biota.LoadTile(data_dir, latitude, longitude, 2010, lee_filter = True, forest_threshold = 15., area_threshold = 1, output_dir = output_dir)


In [59]:
gamma0_2018 = tile_2018.getGamma0()

In [ ]:
fig = plt.figure(figsize = (7, 6))
ax = fig.add_subplot(1, 1, 1)

# Plot map
buildMap(fig, ax, data, lat, lon, title = title, cbartitle = cbartitle, vmin = vmin, vmax = vmax, cmap = cmap, big_labels = big_labels)

# Show in window
plt.show()

# Tidy up
plt.close()

In [61]:
tile_2018.__showArray(gamma0_2018)

AttributeError: 'LoadTile' object has no attribute '__showArray'

In [ ]:
gamma0_2018 = tile_2018.getGamma0(polarisation = 'HV', units = 'decibels', show = True)

In [ ]:
agb_2018 = tile_2018.getAGB(show = True)

In [ ]:
woodycover_2018 = tile_2018.getWoodyCover(show = True)

In [ ]:
tile_2018 = biota.LoadTile(data_dir, latitude, longitude, 2018, lee_filter = True, forest_threshold = 15., area_threshold = 1, output_dir = output_dir)
tile_2015 = biota.LoadTile(data_dir, latitude, longitude, 2015, lee_filter = True, forest_threshold = 15., area_threshold = 1, output_dir = output_dir)
#tile_2010 = biota.LoadTile(data_dir, latitude, longitude, 2010, lee_filter = True, forest_threshold = 15., area_threshold = 1, output_dir = output_dir)

In [ ]:
tile_2018.getAGB(show = True)

In [ ]:
tile_change = biota.LoadChange(tile_2015, tile_2018, change_area_threshold = 2, change_magnitude_threshold = 15)
tile_change.getChangeType(show = True)

In [ ]:
agb_change = tile_change.getAGBChange(show = True, output = True)

In [ ]:
change_type = tile_change.getChangeType(output = True)

In [ ]:
type(tile_change)